# Image Vizualization

The Python API  a difference than the Code Editor do not support interactive vizualization (**Map.addLayer**). To achieve desirable visualization effects you could use **ee.Image.getThumbURL** in replacement of Map.addLayer. Both  ee.Image.getThumbURL and Map.addLayer have the same parameters:



Parameter | Description | Type
--------------|-------------|-------------
bands	| Comma-delimited list of three band names to be mapped to RGB| list
min|	Value(s) to map to 0	number or list of three numbers, one for each |band
max|	Value(s) to map to 255	|number or list of three numbers, one for each band
gain|	Value(s) by which to multiply each pixel value|	number or list of three numbers, one for each band
bias|	Value(s) to add to each DN	| number or list of three numbers, one for each band
gamma|	Gamma correction factor(s)	| number or list of three numbers, one for each band
palette|	List of CSS-style color strings (single-band images only)	|comma-separated list of hex strings
opacity|	The opacity of the layer (0.0 is fully transparent and 1.0 is fully opaque)	| number
format|	Either "jpg" or "png"	|string

In [ ]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
mapid = image.getMapId({
    'bands': ['B4', 'B3', 'B2'], 
    'min': 0, 
    'max': 0.3})

# Display the map with folium!
Mapdisplay(center, {'Median Composite':mapid},zoom_start=8)

## Masking

You can use **`image.updateMask()`** to set the opacity of individual pixels based on where pixels in a mask image are non-zero. Pixels equal to zero in the mask are excluded from computations and the opacity is set to 0 for display. The following example uses an NDWI threshold (see the [Relational Operations section]() for information on thresholds) to update the mask on the NDWI layer created previously:



In [ ]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])

# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
ndwiId = ndwiMasked.getMapId({'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']})

# Display the map with folium!
center=[38., -122.5]
Mapdisplay(center,{'NDWI masked':ndwiId})

## Clipping

The **`image.clip()`** method is useful for achieving cartographic effects. The following example clips the Image **RGB_SF**.

In [ ]:
# Create a circle by drawing a 20000 meter buffer around a point.
roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)

mapRoi = image.clip(roi)
mapId = mapRoi.getMapId()

# Display map
center=[38., -122.5]
Mapdisplay(center,{'Mosaic':mapId})

## Rendering categorical maps

Palettes are also useful for rendering discrete valued maps, for example a land cover map. In the case of multiple classes, use the palette to supply a different color for each class. (The **`image.remap()`** method may be useful in this context, to convert arbitrary labels to consecutive integers). The following example uses a palette to render land cover categories:



In [ ]:
cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01').select('Land_Cover_Type_1')

# Define a palette for the 18 distinct land cover classes.
igbpPalette = [
  'aec3d4', # water
  '152106', '225129', '369b47', '30eb5b', '387242', # forest
  '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # shrub, grass
  '111149', # wetlands
  'cdb33b', # croplands
  'cc0013', # urban
  '33280d', # crop mosaic
  'd7cdcc', # snow and ice
  'f7e084', # barren
  '6f6f6f'  # tundra
]

# Specify the min and max labels and the color palette matching the labels.
vizParams = {'min': 0,
             'max': 17,
             'palette': igbpPalette}
mapId = cover.getMapId(vizParams)

#Display the map with folium!
center = [38., -122.5]
Mapdisplay(center,{'Mosaic':mapId},zoom_start=8)

# Image information and metadata

To explore image bands and properties in the Python API, getInfo() the image and inspect the output in the console. This information can also be accessed programmatically. For example, the following demonstrates how to access information about bands, projections and other metadata:




In [ ]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Get information about the bands as a list.
bandNames = image.bandNames()
print('Band names: ', bandNames.getInfo()) # ee.List of band names

In [ ]:
# Get projection information from band 1.
b1proj = image.select('B1').projection()
print('Band 1 projection: \n', b1proj.getInfo()) # ee.Projection object

In [ ]:
# Get scale (in meters) information from band 1.
b1scale = image.select('B1').projection().nominalScale()
print('Band 1 scale: \n', b1scale.getInfo()) # ee.Number

In [ ]:
# Note that different bands can have different projections and scale.
b8scale = image.select('B8').projection().nominalScale()
print('Band 8 scale: \n', b8scale.getInfo()) # ee.Number

In [ ]:
# Get a list of all metadata properties.
properties = image.propertyNames()
print('Metadata properties:', ) 
properties.getInfo() # ee.List of metadata properties

In [ ]:
# Get a specific metadata property.
cloudiness = image.get('CLOUD_COVER')
print('CLOUD_COVER: ')
cloudiness.getInfo() # ee.Number

In [ ]:
from datetime import datetime as dt

# Get the timestamp and convert it to a date.
date = ee.Date(image.get('system:time_start'))
# We divide by 1000 because Earth Engine returns the timestamp in milisecond and Python in seconds.
tmstp = date.getInfo()['value']/1000 

print('Timestamp:', dt.utcfromtimestamp(tmstp).strftime('%Y-%m-%d %H:%M:%S'))

Note that the results of these queries are **server-side objects**. When you use the method \*.getInfo(), you request that information describing the object be sent from the server to your client. (Learn more about client vs. server in Earth Engine on this [page](https://developers.google.com/earth-engine/client_server)). 



Earth Engine supports many basic mathematical operators. They share some common features. **Earth Engine performs math operations per pixel**. When an operator is applied to an image, it's applied to each unmasked pixel of each band. In the case of operations on two images, the operation is only applied at the locations where pixels in both images are unmasked. Earth Engine automatically matches bands between images. When an operator is applied to two images, the images are expected to have the same number of bands so they can be matched pairwise. However, if one of the images has only a single band, it is matched with all of the bands in the other image, essentially replicating that band enough times to match the other image.


For a simple example, consider the task of creating the Normalized Difference Vegetation Index (NDVI) using Landsat imagery:



In [ ]:
# Load two 5-year Landsat 7 composites.
landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Compute NDVI the hard way.
ndvi1999 = landsat1999.select('B4')\
                      .subtract(landsat1999.select('B3'))\
                      .divide(landsat1999.select('B4').add(landsat1999.select('B3')))

# Compute NDVI the easy way.
ndvi2008 = landsat2008.normalizedDifference(['B4', 'B3'])

#Vizualization
ndwiViz = {'min': 0, 'max': 1, 'palette': ['FF0000', '00FF00']}
Peru = ee.Geometry.Rectangle(-85, -20, -65,0) 

ndvi1999 = ndvi1999.clip(Peru)
ndvi2008 = ndvi2008.clip(Peru)

tiles = {
    "NDVI PERU 1999": ndvi1999.getMapId(ndwiViz), 
    "NDVI PERU 2008": ndvi2008.getMapId(ndwiViz)
}
center = [-12., -75.]
Mapdisplay(center, tiles)

####  Expressions

To implement more complex mathematical expressions, it may be more convenient to use **`image.expression()`**, which parses a text representation of a math operation. The following example uses **`expression()`** to compute the Enhanced Vegetation Index (EVI):

In [ ]:
# Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Compute the EVI using an expression.
evi = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')})

#Display the map with folium!
Eviviz = {'min': 0, 'max': 1, 'palette': ['FF0000', '00FF00']}
tiles = {"EVI 2014": evi.getMapId(Eviviz)}

center  = [37,-122]
Mapdisplay(center, tiles)

Observe that the first argument to expression is the textual representation of the math operation, the second argument is a dictionary where the keys are variable names used in the expression and the values are the image bands to which the variables should be mapped. Bands in the image may be referred to as `b("band name")` or `b(index)`, for example `b(0)`, instead of providing the dictionary. Note that division functions as it does in Python: dividing two integers results in an integer. For example `10 / 20 = 0`. To change this behavior, multiply one of the operands by `1.0: 10 * 1.0 / 20 = 0.5`. Supported expression operators are listed in the following table.

 _ | _ | Operators for expression()
--------------|-------------|-------------
Arithmetic	|+ - * / % **	|Add, Subtract, Multiply, Divide, Modulus, Exponent
Comparison	| == != < > <= >= |	Equal, Not Equal, Less Than, Greater than, etc.
Logical |	&& \|\| ! ^	| And, Or, Not, Xor
Ternary	| ? :	| If then else

# 5. Relational, conditional and Boolean operations

To perform per-pixel comparisons between images, use relational operators. To extract urbanized areas in an image, this example uses relational operators to threshold spectral indices, combining the thresholds with and():

In [ ]:
# Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])

# Create a binary layer using logical operations.
bare = ndvi.lt(0.2).And(ndwi.lt(0))

bare = bare.updateMask(bare)
BareId = bare.getMapId()

tiles = {"Bare": BareId}
center = [37.7726, -122.3578]
Mapdisplay(center, tiles, "Stamen Terrain")

As illustrated by this example, the output of relational and boolean operators is either true (1) or false (0). To mask the 0's, you can mask the resultant binary image with itself. The result from the previous example should look something.



The binary images that are returned by relational and boolean operators can be used with mathematical operators. This example creates zones of urbanization in a nighttime lights image using relational operators and image.add():



In [ ]:
# Load a 2012 nightlights image.
nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl2012.select('stable_lights')

# Define arbitrary thresholds on the 6-bit stable lights band.
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))

# Display the thresholded image as three distinct zones near Paris.
palette = ['000000', '0000FF', '00FF00', 'FF0000']
center = [48.8683, 2.373]

zonesId = zones.getMapId({'min': 0, 'max': 3, 'palette': palette})
tiles = {"Zonas": zonesId}

Mapdisplay(center, tiles)

Note that the code in the previous example is equivalent to using a ternary operator implemented by expression()


In [ ]:
zonesExp = nl2012.expression("(b('stable_lights') > 62) ? 3: (b('stable_lights') > 55) ? 2: (b('stable_lights') > 30) ? 1: 0")

zonesId = zonesExp.getMapId({'min': 0, 'max': 3, 'palette': palette})
tiles = {"Zonas": zonesId}

Mapdisplay(center, tiles)

Another way to implement conditional operations on images is with the image.where() operator. Consider the need to replace masked pixels with some other data. In the following example, cloudy pixels are replaced by pixels from a cloud-free image using where()

In [ ]:
# Load a cloudy Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20130603')

# Load another image to replace the cloudy pixels.
replacement = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20130416')

# Compute a cloud score band.
cloud = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')

# Set cloudy pixels to the other image.
replaced = image.where(cloud.gt(10), replacement)

# Display the result.
replacedId = replaced.getMapId({'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5})

tiles = {"Clouds replaced": replacedId}

center = [37.5419, -121.9872]
Mapdisplay(center, tiles)

In this example, observe the use of the simpleCloudScore() algorithm. This algorithm ranks pixels by cloudiness on a scale of 0-100, with 100 most cloudy. Learn more about simpleCloudScore() on the Landsat Algorithms page.

# Cumulative Cost Mapping

Use image.cumulativeCost() to compute a cost map where every pixel contains the total cost of the lowest cost path to the nearest source location. This process is useful in a variety of contexts such as habitat analysis (Adriaensen et al. 2003), watershed delineation (Melles et al. 2011) and image segmentation (Falcao et al. 2004). Call the cumulative cost function on an image in which each pixel represents the cost per meter to traverse it. Paths are computed through any of a pixel's eight neighbors. Required inputs include a source image, in which each non-zero pixel represents a potential source (or start of a path), and a maxDistance (in meters) over which to compute paths. The algorithm finds the cumulative cost of all paths less than maxPixels = maxDistance/scale in length, where scale is the pixel resolution, or scale of analysis in Earth Engine.


The following example demonstrates computing least-cost paths across a land cover image.


In [ ]:
# A rectangle representing Bangui, Central African Republic.
geometry = ee.Geometry.Rectangle([18.5229, 4.3491, 18.5833, 4.4066])

# Create a source image where the geometry is 1, everything else is 0.
sources = ee.Image().toByte().paint(geometry, 1)

# Mask the sources image with itself.
sources = sources.updateMask(sources)

# The cost data is generated from classes in ESA/GLOBCOVER.
cover = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(0)

# Classes 60, 80, 110, 140 have cost 1.
# Classes 40, 90, 120, 130, 170 have cost 2.
# Classes 50, 70, 150, 160 have cost 3.
cost = cover.eq(60).Or(cover.eq(80)).Or(cover.eq(110)).Or(cover.eq(140)).multiply(1).add(cover.eq(40).Or(cover.eq(90)).Or(cover.eq(120)).Or(cover.eq(130)).Or(cover.eq(170)).multiply(2).add(cover.eq(50).Or(cover.eq(70)).Or(cover.eq(150)).Or(cover.eq(160)).multiply(3)))

# Compute the cumulative cost to traverse the land cover.
cumulativeCost = cost.cumulativeCost(
  source = sources,
  maxDistance = 80 * 1000 # 80 kilometers
)

dicc = {
    'Globcover': cover.getMapId(),
    'accumulated cost': cumulativeCost.getMapId({'min': 0, 'max': 5e4}),
    "source geometry": geometry.getInfo()
}

# Display the results
center = [4.2, 18.71]
Mapdisplay(center, dicc, "Stamen Terrain", 10)

The top figure in which each output pixel represents the accumulated cost to the nearest source. Note that discontinuities can appear in places where the least cost path to the nearest source exceeds maxPixels in length.

The cumulative cost to the source pixels, where cost is determined by the land cover categories. Low costs are black, higher costs are white.